In [1]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer

# Sales

## 加载与查看数据

In [2]:
original_data = pd.read_csv('ruc_Class25Q2_train_price.csv')
original_test = pd.read_csv('ruc_Class25Q2_test_price.csv')
def clean_column_name(col_name):
    return col_name.replace(' ', '')
original_data = original_data.rename(columns=clean_column_name)
original_test = original_test.rename(columns=clean_column_name)
print("=== 数据基本信息 ===")
original_data.info()

/tmp/ipykernel_1743/2369846845.py:1: DtypeWarning: Columns (3,32,34,43,46,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  original_data = pd.read_csv('ruc_Class25Q2_train_price.csv')
/tmp/ipykernel_1743/2369846845.py:2: DtypeWarning: Columns (4,32) have mixed types. Specify dtype option on import or set low_memory=False.
  original_test = pd.read_csv('ruc_Class25Q2_test_price.csv')


=== 数据基本信息 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103871 entries, 0 to 103870
Data columns (total 55 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   城市         103871 non-null  int64  
 1   区域         103871 non-null  float64
 2   板块         103871 non-null  float64
 3   环线         40419 non-null   object 
 4   Price      103871 non-null  float64
 5   房屋户型       103291 non-null  object 
 6   所在楼层       103871 non-null  object 
 7   建筑面积       103871 non-null  object 
 8   套内面积       35984 non-null   object 
 9   房屋朝向       103870 non-null  object 
 10  建筑结构       103291 non-null  object 
 11  装修情况       103291 non-null  object 
 12  梯户比例       101252 non-null  object 
 13  配备电梯       91520 non-null   object 
 14  别墅类型       1443 non-null    object 
 15  交易时间       103871 non-null  object 
 16  交易权属       103871 non-null  object 
 17  上次交易       78422 non-null   object 
 18  房屋用途       103870 non-null  object 
 19  房屋年限    

In [3]:
print("=== 数值型变量描述性统计 ===")
original_data.describe()

=== 数值型变量描述性统计 ===


,城市,区域,板块,Price,抵押信息,lon,lat,年份,区县,板块_comm,容积率,停车位,coord_x,coord_y
count,103871.000000,103871.000000,103871.000000,1.038710e+05,0.0,103871.000000,103871.000000,103871.000000,96630.000000,96293.000000,70717.000000,69303.000000,96740.000000,96740.000000
mean,3.816272,65.564989,604.388405,2.262366e+06,NaN,114.790564,32.181145,2020.913200,65.827155,595.768374,2.780097,1158.042264,115.022829,32.243050
std,3.356300,33.694763,334.938598,2.532925e+06,NaN,5.837851,5.955405,0.845009,33.826543,331.485164,1.607581,1301.382856,5.864949,5.999335
min,0.000000,3.000000,1.000000,7.455330e+04,NaN,103.507243,23.025853,2015.000000,3.000000,1.000000,0.020000,1.000000,103.507243,23.025853
25%,2.000000,35.000000,314.000000,8.910913e+05,NaN,107.620335,30.379362,2021.000000,35.000000,303.000000,1.800000,312.000000,107.637749,30.358083
50%,3.000000,68.000000,602.000000,1.479407e+06,NaN,115.552921,32.136038,2021.000000,68.000000,587.000000,2.500000,734.000000,117.031933,32.170672
75%,6.000000,88.000000,909.000000,2.680757e+06,NaN,118.046991,37.699695,2021.000000,88.000000,895.000000,3.200000,1515.000000,121.574128,40.479189
max,11.000000,131.000000,1186.000000,5.622643e+07,NaN,122.966669,42.189559,2022.000000,131.000000,1186.000000,30.000000,8700.000000,122.966669,42.189559


In [4]:
print(original_data.head(10))

   城市     区域      板块    环线         Price      房屋户型        所在楼层     建筑面积  \
0   0  109.0   150.0  二至三环  6.194049e+06  2室1厅1厨1卫   中楼层 (共5层)    52.3㎡   
1   0   65.0   299.0  五至六环  4.354153e+06  3室1厅1厨1卫    顶层 (共6层)  127.44㎡   
2   0   62.0   911.0  五至六环  3.321992e+06  3室2厅1厨2卫   低楼层 (共6层)  118.02㎡   
3   0  123.0  1102.0   六环外  7.895656e+06  6室3厅1厨3卫    底层 (共2层)  293.23㎡   
4   0   81.0   295.0  三至四环  1.902960e+06     1房间1卫  中楼层 (共10层)   39.85㎡   
5   0  123.0    78.0  五至六环  1.645915e+07  5室2厅1厨4卫   地下室 (共2层)  259.72㎡   
6   0  112.0  1126.0   六环外  3.631467e+06  3室2厅1厨2卫  高楼层 (共20层)   134.1㎡   
7   0  112.0  1126.0   六环外  2.098983e+05       NaN   地下室 (共0层)   12.74㎡   
8   0   81.0   878.0  三至四环  1.189620e+07  3室1厅1厨2卫  高楼层 (共22层)  192.12㎡   
9   0   28.0   547.0  四至五环  1.093772e+07  5室3厅1厨4卫    底层 (共3层)  288.96㎡   

      套内面积     房屋朝向  ...     供水        供暖     供电            燃气费       供热费  \
0      NaN      南 北  ...     民水      集中供暖     民电       2.61元/m³     30元/㎡   
1   123.7㎡      南 北 

## 数据预处理

### 删除部分行/列

In [5]:
# 删除缺失值过多、相关性低、难以结构化、重复的列
missing_counts = original_data.isnull().sum()
missing_sorted = missing_counts.sort_values(ascending=False)
print("=== 列名及缺失值数量（从高到低） ===")
for col, count in missing_sorted.items():
    print(f"{col}: {count}")

=== 列名及缺失值数量（从高到低） ===
抵押信息: 103871
别墅类型: 102428
环线位置: 75774
户型介绍: 75100
供暖: 72926
供热费: 72488
物业办公电话: 68935
套内面积: 67887
环线: 63452
房屋年限: 44510
交通出行: 43264
周边配套: 43138
物业公司: 37908
开发商: 36141
停车费用: 35553
建筑年代: 35101
停车位: 34568
容积率: 33154
绿化率: 32883
燃气费: 32701
物业费: 31158
供水: 30298
供电: 30292
建筑结构_comm: 29119
物业类别: 28413
产权描述: 28413
上次交易: 25449
房屋优势: 21236
核心卖点: 18614
配备电梯: 12351
板块_comm: 7578
区县: 7241
房屋总数: 7131
coord_y: 7131
coord_x: 7131
楼栋总数: 7131
梯户比例: 2619
建筑结构: 580
装修情况: 580
房屋户型: 580
房屋用途: 1
房屋朝向: 1
产权所属: 0
板块: 0
区域: 0
Price: 0
所在楼层: 0
城市: 0
交易时间: 0
交易权属: 0
建筑面积: 0
lat: 0
lon: 0
年份: 0
客户反馈: 0


In [6]:
columns_to_drop = ['抵押信息', '户型介绍', '供暖', '供热费', '物业办公电话', '交通出行', '周边配套', '物业公司', '开发商', '核心卖点', '客户反馈', '上次交易', '区县', '板块_comm', 'coord_x', 'coord_y', '物业类别', '产权描述', '燃气费', '停车费用']
data_1 = original_data.drop(columns_to_drop, axis=1)
test_1 = original_test.drop(columns_to_drop, axis=1)

In [7]:
# 删除缺失值过多的样本
sample_missing_counts = data_1.isnull().sum(axis=1)
print("=== 样本统计 ===")
print(f"总样本数量: {len(data_1)}")
print(f"总列数: {data_1.shape[1]}")
missing_value_distribution = sample_missing_counts.value_counts().sort_index()

print("\n=== 缺失值统计 ===")
print("缺失值数量 | 样本数量")
print("-" * 20)
for missing_count, sample_count in missing_value_distribution.items():
    print(f"{missing_count:^11} | {sample_count:^8}")

=== 样本统计 ===
总样本数量: 103871
总列数: 35

=== 缺失值统计 ===
缺失值数量 | 样本数量
--------------------
     1      |   8516  
     2      |  12987  
     3      |  17454  
     4      |  12367  
     5      |   9252  
     6      |   7570  
     7      |   3607  
     8      |   1137  
     9      |   914   
    10      |   508   
    11      |   2080  
    12      |   9992  
    13      |   8292  
    14      |   4523  
    15      |   2454  
    16      |   1576  
    17      |   485   
    18      |    63   
    19      |    46   
    20      |    6    
    21      |    42   


In [8]:
threshold = 20
data_2 = data_1.copy()
test_2 = test_1.copy()
data_2 = data_2[sample_missing_counts < threshold]
data_2 = data_2.reset_index(drop=True)
print("=== 处理后样本量统计 ===")
print(f"原样本数量: {len(data_1)}")
print(f"删除样本数量: {len(data_1) - len(data_2)}  删除比例：{(len(data_1) - len(data_2))/len(data_1)*100:.2f}%")
print(f"处理后样本数量: {len(data_2)}")

=== 处理后样本量统计 ===
原样本数量: 103871
删除样本数量: 48  删除比例：0.05%
处理后样本数量: 103823


### 数据提取与编码

In [9]:
data_3 = pd.DataFrame()
test_3 = pd.DataFrame()

In [10]:
# 城市、区域、板块
data_3[['城市', '区域', '板块']] = data_2[['城市', '区域', '板块']].copy()
test_3[['城市', '区域', '板块']] = test_2[['城市', '区域', '板块']].copy()

In [11]:
# 环线&环线位置
data_2['环线'] = data_2['环线'].fillna(data_2['环线位置'])
data_3['环线'] = np.nan
test_2['环线'] = test_2['环线'].fillna(test_2['环线位置'])
test_3['环线'] = np.nan

# 定义汉字数字到阿拉伯数字的映射字典
hanzi_to_num = {
    '一': 1, '二': 2, '三': 3, '四': 4, '五': 5,
    '六': 6, '七': 7, '八': 8, '九': 9, '十': 10
}

def process_ring_value(s):
    if pd.isna(s):
        return np.nan
    
    # 转换为字符串并去除首尾空格（避免格式问题）
    s = str(s).strip()
    
    # 内环内→1，外环外→6
    if s == '内环内':
        return 1
    if s == '外环外':
        return 6

    # 内环至中环→2，内环至外环→3，中环至外环→4
    if s == '内环至中环':
        return 2
    if s == '内环至外环':
        return 3 
    if s == '中环至外环':
        return 4
    
    # x环内→x-1，x环外→x
    if '环内' in s:
        x_hanzi = s[0]
        if x_hanzi in hanzi_to_num:
            return hanzi_to_num[x_hanzi] - 1
    elif '环外' in s:
        x_hanzi = s[0]
        if x_hanzi in hanzi_to_num:
            return hanzi_to_num[x_hanzi]
    
    # x至y环→(x+y)/2向下取整
    first_char = s[0] if len(s) > 0 else ''
    third_char = s[2] if len(s) > 2 else ''
    if first_char in hanzi_to_num and third_char in hanzi_to_num:
        return int((hanzi_to_num[first_char] + hanzi_to_num[third_char])/2)
    
    # 其他未覆盖的情况返回缺失值
    return np.nan

data_3['环线'] = data_2['环线'].apply(process_ring_value)
test_3['环线'] = test_2['环线'].apply(process_ring_value)

In [12]:
# Price
data_3['Price'] = data_2['Price'].copy()

In [13]:
# 房屋户型
data_2['房屋户型'] = data_2['房屋户型'].str.replace('房间', '室')
data_3[['室', '厅', '厨', '卫']] = [np.nan, np.nan, np.nan, np.nan]
test_2['房屋户型'] = test_2['房屋户型'].str.replace('房间', '室')
test_3[['室', '厅', '厨', '卫']] = [np.nan, np.nan, np.nan, np.nan]

keywords = ['室', '厅', '厨', '卫']

def process_house_layout(s, keyword):
    if pd.isna(s):
        return np.nan

    # 正则匹配
    pattern = r'(\d+)' + keyword
    match = re.search(pattern, str(s))
    if match:
        return int(match.group(1))
    else:
        return np.nan

for keyword in keywords:
    data_3[keyword] = data_2['房屋户型'].apply(lambda x: process_house_layout(x, keyword))
    test_3[keyword] = test_2['房屋户型'].apply(lambda x: process_house_layout(x, keyword))

In [14]:
# 所在楼层
floor_to_num = {
    '低楼层': 1/3, '中楼层': 2/3, '高楼层': 1.0
}

def process_floor(s):
    if pd.isna(s):
         return pd.Series([np.nan, np.nan], index=['所在楼层', '总层数'])

    floor = np.nan
    floor_tol = np.nan
    
    pattern = r'共\s*(\d+)\s*层'
    match = re.search(pattern, str(s))
    if match:
        floor_tol = int(match.group(1))

    s_1 = str(s).split('(')[0].strip()
    if s_1 == '地下室':
        floor = -1
    elif s_1 == '底层':
        floor = 1
    elif s_1 == '顶层':
        floor = floor_tol
    elif s_1 in floor_to_num:
        floor = floor_to_num[s_1] * floor_tol
    return pd.Series([floor, floor_tol], index=['所在楼层', '总层数'])
    
data_3 = pd.concat([data_3, data_2['所在楼层'].apply(process_floor)], axis=1)
test_3 = pd.concat([test_3, test_2['所在楼层'].apply(process_floor)], axis=1)

In [15]:
# 建筑面积&套内面积
data_3[['建筑面积', '套内面积']] = [np.nan, np.nan]
test_3[['建筑面积', '套内面积']] = [np.nan, np.nan]

def process_area(s):
    if pd.isna(s):
        return np.nan
    pattern = r'(\d+\.?\d*)\s*㎡'
    match = re.search(pattern, str(s))
    if match:
        return float(match.group(1))
    else:
        return np.nan

data_3['建筑面积'] = data_2['建筑面积'].apply(process_area)
data_3['套内面积'] = data_2['套内面积'].apply(process_area)
test_3['建筑面积'] = test_2['建筑面积'].apply(process_area)
test_3['套内面积'] = test_2['套内面积'].apply(process_area)

In [16]:
# 房屋朝向
directions = ['东', '南', '西', '北', '东南', '东北', '西南', '西北']

def process_directions(s):
    if pd.isna(s):
        return pd.Series(np.nan, index=[f'房屋朝向_{d}' for d in directions])
    
    parts = [p.strip() for p in str(s).split() if p.strip()]
    result = pd.Series(0, index=[f'房屋朝向_{d}' for d in directions])
    
    for direction in directions:
        col_name = f'房屋朝向_{direction}'
        result[col_name] = parts.count(direction)
    return result

data_3 = pd.concat([data_3, data_2['房屋朝向'].apply(process_directions)], axis=1)
test_3 = pd.concat([test_3, test_2['房屋朝向'].apply(process_directions)], axis=1)

In [17]:
# 建筑结构、装修情况
data_3['建筑结构'] = data_2['建筑结构'].fillna('未知结构')
data_3['装修情况'] = data_2['装修情况'].copy()
test_3['建筑结构'] = test_2['建筑结构'].fillna('未知结构')
test_3['装修情况'] = test_2['装修情况'].copy()

In [18]:
# 梯户比例&配备电梯
def chinese_to_num(chinese):
    # 汉字数字映射表
    num_map = {
        '零': 0, '一': 1, '二': 2, '三': 3, '四': 4,
        '五': 5, '六': 6, '七': 7, '八': 8, '九': 9,
        '十': 10, '百': 100, '千': 1000, '万': 10000, '两': 2
    }
    
    result = 0  # 最终结果
    current = 0  # 当前累计的数字（用于处理单位前的数字）
    
    for char in chinese:
        if char == '零':  # 忽略"零"（仅占位，不影响数值）
            continue
        if char in ['十', '百', '千', '万']:  # 处理单位
            unit = num_map[char]
            # 若单位前无数字（如"十"默认是10），则current设为1
            current = current if current != 0 else 1
            result += current * unit
            current = 0  # 重置当前累计，准备处理下一段
        else:  # 处理单个数字
            current = num_map[char]
    
    # 加上末尾无单位的数字
    result += current
    return result

def process_lift(s):
    if pd.isna(s):
        return pd.Series(np.nan, index=['梯数', '户数'])
    
    s_lift = s.split('梯')[0]
    s_house = s.split('梯')[1].split('户')[0]
    num_lift = chinese_to_num(s_lift)
    num_house = chinese_to_num(s_house)
    return pd.Series([num_lift, num_house], index=['梯数', '户数'])
    
data_3 = pd.concat([data_3, data_2['梯户比例'].apply(process_lift)], axis=1)
test_3 = pd.concat([test_3, test_2['梯户比例'].apply(process_lift)], axis=1)

# 合并配备电梯内容
cond_empty_data = pd.isna(data_3['梯数'])
cond_empty_test = pd.isna(test_3['梯数'])
# 若配备电梯为"有"，梯数设为1
data_3.loc[cond_empty_data  & (data_2['配备电梯'] == '有'), '梯数'] = 1
test_3.loc[cond_empty_test  & (test_2['配备电梯'] == '有'), '梯数'] = 1
# 若配备电梯为"无"，梯数设为0
data_3.loc[cond_empty_data  & (data_2['配备电梯'] == '无'), '梯数'] = 0
test_3.loc[cond_empty_test  & (test_2['配备电梯'] == '无'), '梯数'] = 0

In [19]:
# 别墅类型
data_3['别墅类型'] = data_2['别墅类型'].fillna('非别墅')
test_3['别墅类型'] = test_2['别墅类型'].fillna('非别墅')

In [20]:
# 交易时间
data_3['交易时间'] = data_2['交易时间'].str.split('-').str[0].astype('Int64')
test_3['交易时间'] = test_2['交易时间'].str.split('-').str[0].astype('Int64')

In [21]:
# 交易权属、房屋用途、房屋年限、产权所属
data_3[['交易权属', '房屋用途', '房屋年限', '产权所属']] = data_2[['交易权属', '房屋用途', '房屋年限', '产权所属']].copy()
test_3[['交易权属', '房屋用途', '房屋年限', '产权所属']] = test_2[['交易权属', '房屋用途', '房屋年限', '产权所属']].copy()

In [22]:
# 房屋优势（装修/地铁）
data_3[['装修', '地铁']] = 0
test_3[['装修', '地铁']] = 0

def process_advantage(s, item):
    if pd.isna(s):
        return 0

    if item in s:
        return 1
    else:
        return 0

data_3['装修'] = data_2['房屋优势'].apply(lambda x: process_advantage(x, '装修'))
data_3['地铁'] = data_2['房屋优势'].apply(lambda x: process_advantage(x, '地铁'))
test_3['装修'] = test_2['房屋优势'].apply(lambda x: process_advantage(x, '装修'))
test_3['地铁'] = test_2['房屋优势'].apply(lambda x: process_advantage(x, '地铁'))

In [23]:
# lon、lat、年份
data_3[['lon', 'lat', '年份']] = data_2[['lon', 'lat', '年份']].copy()
test_3[['lon', 'lat', '年份']] = test_2[['lon', 'lat', '年份']].copy()

In [24]:
# 建筑年代
def process_construct_year(s):
    if pd.isna(s):
        return pd.Series(np.nan, index=['建筑年代_1', '建筑年代_2'])

    s_year = s.split('年')[0]
    s_len = len(s_year.split('-'))
    left = 0
    right = 0
    if s_len > 1:
        left = int(s_year.split('-')[0])
        right = int(s_year.split('-')[1])
    else:
        left = int(s_year)
        right = int(s_year)
    return pd.Series([left, right], index=['建筑年代_1', '建筑年代_2'])
    
data_3 = pd.concat([data_3, data_2['建筑年代'].apply(process_construct_year)], axis=1)
test_3 = pd.concat([test_3, test_2['建筑年代'].apply(process_construct_year)], axis=1)

In [25]:
# 房屋总数、楼栋总数、绿化率、容积率
data_3['房屋总数'] = data_2['房屋总数'].str.replace('户', '').astype('Int64')
data_3['楼栋总数'] = data_2['楼栋总数'].str.replace('栋', '').astype('Int64')
data_3['绿化率'] = data_2['绿化率'].str.replace('%', '').astype('Float64')/100
data_3['容积率'] = data_2['容积率']/100

test_3['房屋总数'] = test_2['房屋总数'].str.replace('户', '').astype('Int64')
test_3['楼栋总数'] = test_2['楼栋总数'].str.replace('栋', '').astype('Int64')
test_3['绿化率'] = test_2['绿化率'].str.replace('%', '').astype('Float64')/100
test_3['容积率'] = test_2['容积率']/100

In [26]:
# 建筑结构_comm（塔楼/平房/板楼/塔板结合）
structures = ['塔楼', '平房', '板楼', '塔板结合']

def process_structure(s, item):
    if pd.isna(s):
        return np.nan

    if item in s:
        return 1
    else:
        return 0

for structure in structures:
    data_3[structure] = data_2['建筑结构_comm'].apply(lambda x: process_structure(x, structure))
    test_3[structure] = test_2['建筑结构_comm'].apply(lambda x: process_structure(x, structure))

In [27]:
# 供水（民水/商水）
waters = ['民水', '商水']

def process_water(s, item):
    if pd.isna(s):
        return np.nan

    if item in s:
        return 1
    else:
        return 0

for water in waters:
    data_3[water] = data_2['供水'].apply(lambda x: process_water(x, water))
    test_3[water] = test_2['供水'].apply(lambda x: process_water(x, water))

In [28]:
# 供电（民电/商电）
electrics = ['民电', '商电']

def process_electric(s, item):
    if pd.isna(s):
        return np.nan

    if item in s:
        return 1
    else:
        return 0

for electric in electrics:
    data_3[electric] = data_2['供电'].apply(lambda x: process_electric(x, electric))
    test_3[electric] = test_2['供电'].apply(lambda x: process_electric(x, electric))

In [29]:
# 物业费
def process_cost(s):
    if pd.isna(s):
        return np.nan

    num = np.nan
    numbers = re.findall(r'\d+', s)
    if numbers:
        numbers_list = [int(x) for x in numbers]
        num = sum(numbers_list) / len(numbers_list)
    return num

data_3['物业费'] = data_2['物业费'].apply(process_cost)
test_3['物业费'] = test_2['物业费'].apply(process_cost)

In [30]:
# 停车位
data_3['停车位'] = data_2['停车位'].copy()
test_3['停车位'] = test_2['停车位'].copy()

In [31]:
print(data_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103823 entries, 0 to 103822
Data columns (total 52 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   城市       103823 non-null  int64  
 1   区域       103823 non-null  float64
 2   板块       103823 non-null  float64
 3   环线       40419 non-null   float64
 4   Price    103823 non-null  float64
 5   室        103291 non-null  float64
 6   厅        102572 non-null  float64
 7   厨        102572 non-null  float64
 8   卫        103291 non-null  float64
 9   所在楼层     103823 non-null  float64
 10  总层数      103823 non-null  float64
 11  建筑面积     103823 non-null  float64
 12  套内面积     35984 non-null   float64
 13  房屋朝向_东   103822 non-null  float64
 14  房屋朝向_南   103822 non-null  float64
 15  房屋朝向_西   103822 non-null  float64
 16  房屋朝向_北   103822 non-null  float64
 17  房屋朝向_东南  103822 non-null  float64
 18  房屋朝向_东北  103822 non-null  float64
 19  房屋朝向_西南  103822 non-null  float64
 20  房屋朝向_西北  103822 non-null  

In [32]:
print(test_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34017 entries, 0 to 34016
Data columns (total 51 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   城市       34017 non-null  int64  
 1   区域       34017 non-null  float64
 2   板块       34017 non-null  float64
 3   环线       15677 non-null  float64
 4   室        34003 non-null  float64
 5   厅        33853 non-null  float64
 6   厨        33853 non-null  float64
 7   卫        34003 non-null  float64
 8   所在楼层     34017 non-null  float64
 9   总层数      34017 non-null  float64
 10  建筑面积     34017 non-null  float64
 11  套内面积     9915 non-null   float64
 12  房屋朝向_东   34017 non-null  int64  
 13  房屋朝向_南   34017 non-null  int64  
 14  房屋朝向_西   34017 non-null  int64  
 15  房屋朝向_北   34017 non-null  int64  
 16  房屋朝向_东南  34017 non-null  int64  
 17  房屋朝向_东北  34017 non-null  int64  
 18  房屋朝向_西南  34017 non-null  int64  
 19  房屋朝向_西北  34017 non-null  int64  
 20  建筑结构     34017 non-null  object 
 21  装修情况     340

In [33]:
data_3.to_feather('Change_11/data_3.feather')
test_3.to_feather('Change_11/test_3.feather')

### 填充缺失值

In [34]:
'''
# 提取数值型特征和非数值型特征
numerical_cols = ['环线', '室', '厅', '厨', '卫', '所在楼层', '总层数', '建筑面积', '套内面积', '梯数', '户数', 'lon', 'lat', '年份', '建筑年代_1', '建筑年代_2',
                  '房屋总数', '楼栋总数', '绿化率', '容积率',  '物业费', '停车位']
categorical_cols = ['城市', '区域', '板块', '房屋朝向_东', '房屋朝向_南', '房屋朝向_西', '房屋朝向_北', '房屋朝向_东南', '房屋朝向_东北', '房屋朝向_西南', '房屋朝向_西北',
                    '建筑结构', '装修情况', '别墅类型', '交易时间', '交易权属', '房屋用途', '房屋年限', '产权所属', '装修', '地铁', '塔楼', '平房', '板楼', '塔板结合', '民水',
                    '商水', '民电', '商电']
'''

"\n# 提取数值型特征和非数值型特征\nnumerical_cols = ['环线', '室', '厅', '厨', '卫', '所在楼层', '总层数', '建筑面积', '套内面积', '梯数', '户数', 'lon', 'lat', '年份', '建筑年代_1', '建筑年代_2',\n                  '房屋总数', '楼栋总数', '绿化率', '容积率',  '物业费', '停车位']\ncategorical_cols = ['城市', '区域', '板块', '房屋朝向_东', '房屋朝向_南', '房屋朝向_西', '房屋朝向_北', '房屋朝向_东南', '房屋朝向_东北', '房屋朝向_西南', '房屋朝向_西北',\n                    '建筑结构', '装修情况', '别墅类型', '交易时间', '交易权属', '房屋用途', '房屋年限', '产权所属', '装修', '地铁', '塔楼', '平房', '板楼', '塔板结合', '民水',\n                    '商水', '民电', '商电']\n"

In [35]:
# 提取数值型特征和非数值型特征
numerical_cols = data_3.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols.remove('Price')
numerical_cols.remove('城市')
numerical_cols.remove('区域')
numerical_cols.remove('板块')
categorical_cols = data_3.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_cols.append('城市')
categorical_cols.append('区域')
categorical_cols.append('板块')

In [36]:
# 数值型特征填充缺失值：InterativeImputer
imputer = IterativeImputer(
    max_iter=30,
    tol=1e-3,
    initial_strategy='median',  # 初始化策略：median, mean, most_frequent
    imputation_order='ascending',  # 填补顺序：ascending, descending, roman, random
    random_state=42,
    verbose=2
)

In [37]:
data_num = pd.DataFrame(imputer.fit_transform(data_3[numerical_cols]), columns=numerical_cols)
test_num = pd.DataFrame(imputer.transform(test_3[numerical_cols]), columns=numerical_cols)
print(data_num.info())

[IterativeImputer] Completing matrix with shape (103823, 41)
[IterativeImputer] Ending imputation round 1/30, elapsed time 9.79
[IterativeImputer] Change: 4615.419908260661, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 2/30, elapsed time 19.18
[IterativeImputer] Change: 1825.021844735768, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 3/30, elapsed time 27.68
[IterativeImputer] Change: 1092.2058285880767, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 4/30, elapsed time 37.28
[IterativeImputer] Change: 1905.3934780861398, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 5/30, elapsed time 45.67
[IterativeImputer] Change: 491.01057522676757, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 6/30, elapsed time 55.28
[IterativeImputer] Change: 510.08983229729967, scaled tolerance: 12.669 
[IterativeImputer] Ending imputation round 7/30, elapsed time 63.77
[IterativeImputer] Change: 3

In [38]:
# 非数值型变量填充缺失值：基于NearestNeighbors(用Neighbors的众数填充，若众数为空，则用全局众数填充)
scaler_num = StandardScaler()
data_num_standard = scaler_num.fit_transform(data_num)
test_num_standard = scaler_num.transform(test_num)
k = 100
nn = NearestNeighbors(n_neighbors=k+1)
nn.fit(data_num_standard)
distances_data, indices_data = nn.kneighbors(data_num_standard)
distances_test, indices_test = nn.kneighbors(test_num_standard)

In [39]:
data_cat = data_3[categorical_cols].copy()
test_cat = test_3[categorical_cols].copy()

# 预计算每个非数值型特征的全局众数
global_modes = {}
for col in categorical_cols:
    col_mode = data_cat[col].mode()
    if not col_mode.empty:
        global_modes[col] = col_mode[0]
    else:
        global_modes[col] = np.nan

In [40]:
# 遍历每个非数值型列，填充缺失值
for col in categorical_cols:
    missing_indices = data_cat[data_cat[col].isna()].index
    if len(missing_indices) == 0:
        continue
    
    # 对每个缺失值样本进行填充
    for idx in missing_indices:
        # 获取近邻索引
        neighbor_ids = indices_data[idx][1:]  # indices第一列是样本自身
        # 提取近邻在当前列的非缺失值
        neighbor_values = data_cat.iloc[neighbor_ids][col].dropna()

        # 填充
        if not neighbor_values.empty:
            fill_value = neighbor_values.mode()[0]
        else:
            fill_value = global_modes[col]
        data_cat.loc[idx, col] = fill_value

# 遍历每个非数值型列，填充缺失值
for col in categorical_cols:
    missing_indices = test_cat[test_cat[col].isna()].index
    if len(missing_indices) == 0:
        continue
    
    # 对每个缺失值样本进行填充
    for idx in missing_indices:
        # 获取近邻索引
        neighbor_ids = indices_test[idx]
        # 提取近邻在当前列的非缺失值
        neighbor_values = data_cat.iloc[neighbor_ids][col].dropna()

        # 填充
        if not neighbor_values.empty:
            fill_value = neighbor_values.mode()[0]
        else:
            fill_value = global_modes[col]
        test_cat.loc[idx, col] = fill_value

In [41]:
# 合并数值型与非数值型
data_4 = pd.concat([data_num, data_cat], axis=1)
test_4 = pd.concat([test_num, test_cat], axis=1)
data_4.to_feather('Change_11/data_4.feather')
test_4.to_feather('Change_11/test_4.feather')

### 独热编码分类变量

In [42]:
categorical_cols_2 = [col for col in categorical_cols if col != '城市' and col !='板块']

In [43]:
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
d_cat_encoded = encoder.fit_transform(data_cat[categorical_cols_2])
t_cat_encoded = encoder.transform(test_cat[categorical_cols_2])
data_cat_encoded = pd.DataFrame(d_cat_encoded, columns=encoder.get_feature_names_out(categorical_cols_2),index=data_cat.index)
test_cat_encoded = pd.DataFrame(t_cat_encoded, columns=encoder.get_feature_names_out(categorical_cols_2),index=test_cat.index)

/root/miniconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [3, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [44]:
data_5 = pd.concat([data_num, data_cat_encoded], axis=1)
test_5 = pd.concat([test_num, test_cat_encoded], axis=1)
data_5.to_feather('Change_11/data_5.feather')
test_5.to_feather('Change_11/test_5.feather')

### 处理数值型变量异常值

In [45]:
def process_outliers(df, columns=None):
    df_clean = df.copy()
    
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    
    for col in columns:
        # 计算均值和标准差
        mean = df_clean[col].mean()
        std = df_clean[col].std()
        
        # 计算3σ原则的上下界
        lower_bound = mean - 3 * std
        upper_bound = mean + 3 * std
        
        # 删除异常值
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    
    return df_clean

# 删除异常值
data_combined = pd.concat([data_5, data_3['Price']], axis=1)
data_6 = process_outliers(data_combined, ['Price', '总层数', '建筑面积', '套内面积', '梯数', '户数', '房屋总数', '楼栋总数', '绿化率', '容积率', '停车位'])
print(f"原始数据形状: {data_combined.shape}")
print(f"删除异常值后形状: {data_6.shape}")

原始数据形状: (103823, 207)
删除异常值后形状: (88688, 207)


### 引入交叉项

In [46]:
# 楼层比
data_6['楼层比'] = data_6['所在楼层'] / ( data_6['总层数'] + 1 )
test_5['楼层比'] = test_5['所在楼层'] / ( test_5['总层数'] + 1 )

# 套内面积比
data_6['套内面积比'] = data_6['套内面积'] / data_6['建筑面积']
test_5['套内面积比'] = test_5['套内面积'] / test_5['建筑面积']

# 梯户比
data_6['梯户比'] = data_6['梯数'] / data_6['户数']
test_5['梯户比'] = test_5['梯数'] / test_5['户数']

# 楼房比
data_6['楼房比'] = data_6['楼栋总数'] / data_6['房屋总数']
test_5['楼房比'] = test_5['楼栋总数'] / test_5['房屋总数']

# 容积绿化比
data_6['容积绿化比'] = data_6['容积率'] / data_6['绿化率']
test_5['容积绿化比'] = test_5['容积率'] / test_5['绿化率']

In [47]:
data_6.to_feather('Change_11/data_6.feather')
test_5.to_feather('Change_11/test_6.feather')

## 模型训练与预测

In [48]:
# 划分样本内（训练集）与样本外（测试集）
scaler = StandardScaler()
X = scaler.fit_transform(data_6.drop('Price', axis=1))
X_predict = scaler.transform(test_5)
y = data_6['Price'] / data_6['建筑面积']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=111
)

In [49]:
# 模型定义与超参数网格
models = {
    'OLS': LinearRegression(),
    'Lasso': Lasso(random_state=42),
    'Ridge': Ridge(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
}

param_grids = {
    'Lasso': {'alpha': [0.01, 0.1, 1, 10, 100]},
    'Ridge': {'alpha': [0.01, 0.1, 1, 10, 100]},
    'ElasticNet': {
        'alpha': [0.01, 0.1, 1, 10],
        'l1_ratio': [0.2, 0.5, 0.8]
    },
}

# 调优函数
def tune_model(model_name, X, y):
    model = models[model_name]
    if model_name == 'OLS':
        model.fit(X, y)
        cv_scores = cross_val_score(
            estimator=model,
            X=X,
            y=y,
            cv=6,
            scoring='neg_mean_absolute_error',
            n_jobs=-1
        )
        mae = -cv_scores.mean()
        return model, None, mae
    else:
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grids[model_name],
            cv=6,
            scoring='neg_mean_absolute_error',
            n_jobs=-1
        )
        grid_search.fit(X, y)
        return grid_search.best_estimator_, grid_search.best_params_, -grid_search.best_score_

In [50]:
# 训练并调优所有模型
tuned_models = {}
for name in models.keys():
    print(f"调优{name}模型...")
    model, best_params, mae = tune_model(name, X_train, y_train)
    tuned_models[name] = {'model': model, 'params': best_params, 'mae': mae}
    print(tuned_models[name])

调优OLS模型...
{'model': LinearRegression(), 'params': None, 'mae': np.float64(4479.547271687191)}
调优Lasso模型...


/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.402e+12, tolerance: 2.096e+09
  model = cd_fast.enet_coordinate_descent(
/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.246e+11, tolerance: 2.104e+09
  model = cd_fast.enet_coordinate_descent(
/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.369e+12,

{'model': Lasso(alpha=1, random_state=42), 'params': {'alpha': 1}, 'mae': np.float64(4473.031387153794)}
调优Ridge模型...
{'model': Ridge(alpha=1, random_state=42), 'params': {'alpha': 1}, 'mae': np.float64(4464.526086889843)}
调优ElasticNet模型...


/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.324e+12, tolerance: 2.069e+09
  model = cd_fast.enet_coordinate_descent(
/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.335e+12, tolerance: 2.089e+09
  model = cd_fast.enet_coordinate_descent(
/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.335e+12,

{'model': ElasticNet(alpha=0.01, l1_ratio=0.8, random_state=42), 'params': {'alpha': 0.01, 'l1_ratio': 0.8}, 'mae': np.float64(4479.7683490277395)}


/root/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.599e+12, tolerance: 2.513e+09
  model = cd_fast.enet_coordinate_descent(


In [51]:
# 计算RMAE（MAE除以目标均值）
def rmae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    return mae / np.mean(y_true)

# 评估函数
def evaluate_model(model_name, X_train, y_train, X_test, y_test):
    model = tuned_models[model_name]['model']
    
    # 样本内预测
    y_pred_train = model.predict(X_train)
    mae_in = mean_absolute_error(y_train, y_pred_train)
    rmae_in = rmae(y_train, y_pred_train)
    
    # 样本外预测
    y_pred_test = model.predict(X_test)
    mae_out = mean_absolute_error(y_test, y_pred_test)
    rmae_out = rmae(y_test, y_pred_test)
    
    # 6折交叉验证
    mae_cv = tuned_models[name]['mae']
    rmae_cv = mae_cv / np.mean(y_train)  # 用训练集均值计算RMAE
    
    return {
        'MAE_in': mae_in, 'RMAE_in': rmae_in,
        'MAE_out': mae_out, 'RMAE_out': rmae_out,
        'MAE_cv': mae_cv, 'RMAE_cv': rmae_cv
    }

In [52]:
# 评估所有模型
results = {}
for name in tuned_models.keys():
    results[name] = evaluate_model(name, X_train, y_train, X_test, y_test)

In [53]:
# 整理报告表格（以MAE为例，RMAE类似）
report = pd.DataFrame(columns=['指标', '样本内', '样本外', '交叉验证'])

# 填充OLS结果
report.loc[0] = [
    'OLS (MAE)', 
    round(results['OLS']['MAE_in'], 2),
    round(results['OLS']['MAE_out'], 2),
    round(results['OLS']['MAE_cv'], 2)
]

# 填充Lasso结果
report.loc[1] = [
    'Lasso (MAE)', 
    round(results['Lasso']['MAE_in'], 2),
    round(results['Lasso']['MAE_out'], 2),
    round(results['Lasso']['MAE_cv'], 2)
]

# 填充Ridge结果
report.loc[2] = [
    'Ridge (MAE)', 
    round(results['Ridge']['MAE_in'], 2),
    round(results['Ridge']['MAE_out'], 2),
    round(results['Ridge']['MAE_cv'], 2)
]

# 填充ElasticNet结果
report.loc[3] = [
    'ElasticNet (MAE)', 
    round(results['ElasticNet']['MAE_in'], 2),
    round(results['ElasticNet']['MAE_out'], 2),
    round(results['ElasticNet']['MAE_cv'], 2)
]


print("\n性能指标报告（MAE）：")
print(report)

# 打印RMAE结果（可选）
print("\nRMAE结果：")
for name in ['OLS', 'Lasso', 'Ridge', 'ElasticNet']:
    print(f"{name} - 样本内: {round(results[name]['RMAE_in'], 4)}, "
          f"样本外: {round(results[name]['RMAE_out'], 4)}, "
          f"交叉验证: {round(results[name]['RMAE_cv'], 4)}")


性能指标报告（MAE）：
                 指标      样本内      样本外     交叉验证
0         OLS (MAE)  4460.68  4554.41  4479.55
1       Lasso (MAE)  4456.89  4542.19  4473.03
2       Ridge (MAE)  4447.43  4539.22  4464.53
3  ElasticNet (MAE)  4463.43  4548.29  4479.77

RMAE结果：
OLS - 样本内: 0.199, 样本外: 0.2011, 交叉验证: 0.1998
Lasso - 样本内: 0.1988, 样本外: 0.2006, 交叉验证: 0.1996
Ridge - 样本内: 0.1984, 样本外: 0.2004, 交叉验证: 0.1992
ElasticNet - 样本内: 0.1991, 样本外: 0.2008, 交叉验证: 0.1999


In [55]:
# 查看模型在整个训练数据上的指标
X_all = data_5.copy()
X_all['楼层比'] = X_all['所在楼层'] / ( X_all['总层数'] + 1 )
X_all['套内面积比'] = X_all['套内面积'] / X_all['建筑面积']
X_all['梯户比'] = X_all['梯数'] / X_all['户数']
X_all['楼房比'] = X_all['楼栋总数'] / X_all['房屋总数']
X_all['容积绿化比'] = X_all['容积率'] / X_all['绿化率']
X_all = scaler.transform(X_all)
y_all = data_3['Price']

In [56]:
for name in tuned_models.keys():
    model = tuned_models[name]['model']
    y_pred = model.predict(X_all) * data_5['建筑面积']
    mae1 = mean_absolute_error(y_all, y_pred)
    rmae1 = mae1/np.mean(y_all)
    mse1 = mean_squared_error(y_all, y_pred)
    rmse1 = np.sqrt(mse1)
    r21 = r2_score(y_all, y_pred)
    print('\n', model)
    print('mae:', mae1)
    print('rmae:', rmae1)
    print('mse:', mse1)
    print('rmse:', rmse1)
    print('r2:', r21)


 LinearRegression()
mae: 530444.7809269216
rmae: 0.2343620248179795
mse: 1523925889025.8938
rmse: 1234473.9320965405
r2: 0.7624986129790199

 Lasso(alpha=1, random_state=42)
mae: 529026.6902881196
rmae: 0.23373548157458232
mse: 1482069648291.0466
rmse: 1217402.8290960419
r2: 0.7690218404545794

 Ridge(alpha=1, random_state=42)
mae: 528896.8299981881
rmae: 0.2336781064780857
mse: 1507991005423.9253
rmse: 1228002.8523679923
r2: 0.7649820388363656

 ElasticNet(alpha=0.01, l1_ratio=0.8, random_state=42)
mae: 530530.4436975771
rmae: 0.2343998724905445
mse: 1515960909420.7903
rmse: 1231243.6434032016
r2: 0.7637399421784439


In [57]:
for name in tuned_models.keys():
    model = tuned_models[name]['model']
    y_pred = model.predict(X_predict) * test_5['建筑面积']
    
    # 创建结果DataFrame
    result_df = pd.DataFrame({
        'ID': original_test['ID'],
        'Price': y_pred
    })
    
    # 保存为CSV文件
    filename = f'Change_11/{name}_predictions_sales.csv'
    result_df.to_csv(filename, index=False)
    print(f"{name} 预测结果已保存到 {filename}")

OLS 预测结果已保存到 Change_11/OLS_predictions_sales.csv
Lasso 预测结果已保存到 Change_11/Lasso_predictions_sales.csv
Ridge 预测结果已保存到 Change_11/Ridge_predictions_sales.csv
ElasticNet 预测结果已保存到 Change_11/ElasticNet_predictions_sales.csv
